# MLP for Transfusion Prediction
This notebook will be experimenting with transfusion prediction using an MLP. A subset will be take of the training dataset previously made.


## Get data
We will first import training and testing datasets that have already been split.

In [29]:
# imports here
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score


In [30]:
data_directory = "C:\\Users\\micha\\OneDrive - UT Health San Antonio\\UTHSCSA\\Trauma\\TransfusionPrediction\\trauma_r\\"
train = pd.read_csv(data_directory + "train_trauma.csv")
print(train.shape)
train.head()

(986366, 80)


,onehot__SEX_1.0,onehot__SEX_2.0,onehot__SEX_3.0,onehot__ETHNICITY_1.0,onehot__ETHNICITY_2.0,onehot__TBIMIDLINESHIFT_1.0,onehot__TBIMIDLINESHIFT_2.0,onehot__TBIMIDLINESHIFT_3.0,onehot__TEACHINGSTATUS_1.0,onehot__TEACHINGSTATUS_5.0,...,scaler__RESPIRATORYRATE,scaler__PULSEOXIMETRY,scaler__HEIGHT,scaler__WEIGHT,scaler__TOTALGCS,scaler__HOSPITALARRIVALHRS,scaler__HOSPITALARRIVALDAYS,scaler__TBIHIGHESTTOTALGCS,scaler__ISS,transfusion
0,0,1,0,0,1,0,1,0,0,1,...,1.052371,0.361127,-3.117664,-2.272820,0.325218,-9.088105e-18,-0.021071,9.919297e-01,0.838320,No
1,1,0,0,1,0,0,1,0,0,1,...,-1.052626,0.511366,-0.132069,-0.838981,0.325218,-9.088105e-18,-0.021071,9.919297e-01,-0.032457,No
2,1,0,0,0,1,0,1,0,0,1,...,-0.210627,0.361127,0.329500,-0.815281,0.325218,-9.088105e-18,-0.021071,-2.524544e-15,-1.027631,No
3,1,0,0,0,1,0,1,0,0,1,...,-0.210627,0.511366,-1.540148,-1.640825,0.325218,-9.088105e-18,-0.021071,-2.524544e-15,-0.530044,No
4,0,1,0,0,1,0,1,0,0,1,...,-0.210627,0.511366,-2.971598,-2.027922,0.325218,-9.088105e-18,-0.021071,9.919297e-01,-0.530044,No


In [31]:
# For prototyping, we will use a subset of the data as the entire dataset is too big

sample_size = 0.1  # 10% of the dataset, about 100000

train_sample, _ = train_test_split(train, train_size=sample_size, stratify=train['transfusion'], random_state=42)

train_sample = train


In [32]:
test = pd.read_csv(data_directory + "test_trauma.csv")
print(test.shape)
test.head()

(246590, 80)


,onehot__SEX_1.0,onehot__SEX_2.0,onehot__SEX_3.0,onehot__ETHNICITY_1.0,onehot__ETHNICITY_2.0,onehot__TBIMIDLINESHIFT_1.0,onehot__TBIMIDLINESHIFT_2.0,onehot__TBIMIDLINESHIFT_3.0,onehot__TEACHINGSTATUS_1.0,onehot__TEACHINGSTATUS_5.0,...,scaler__RESPIRATORYRATE,scaler__PULSEOXIMETRY,scaler__HEIGHT,scaler__WEIGHT,scaler__TOTALGCS,scaler__HOSPITALARRIVALHRS,scaler__HOSPITALARRIVALDAYS,scaler__TBIHIGHESTTOTALGCS,scaler__ISS,transfusion
0,0,1,0,0,1,0,1,0,0,1,...,1.495690e-15,2.135025e-15,-5.104166e+00,2.976689,-2.849348e-15,-9.088105e-18,-0.021071,9.919297e-01,-0.405648,No
1,1,0,0,0,1,0,1,0,0,1,...,-2.106272e-01,5.113665e-01,-1.172061e+00,-0.550633,3.252183e-01,-9.088105e-18,-0.021071,-2.524544e-15,-1.027631,No
2,1,0,0,0,1,0,1,0,0,1,...,-8.421263e-01,3.611275e-01,-2.650252e+00,-2.087171,3.252183e-01,-9.088105e-18,-0.021071,9.919297e-01,-0.654441,No
3,0,1,0,1,0,0,1,0,0,1,...,2.103722e-01,3.611275e-01,-1.963383e-01,-1.407777,-7.579178e-02,-9.088105e-18,-0.021071,9.919297e-01,-0.530044,No
4,0,1,0,0,1,0,1,0,0,1,...,2.103722e-01,5.113665e-01,9.963489e-15,-1.881773,3.252183e-01,-9.088105e-18,-0.021071,-2.524544e-15,-0.654441,No


In [33]:
# Convert "Yes" to 1 and "No" to 0 for binary classification
train_sample["transfusion"] = train_sample["transfusion"].map({"Yes": 1, "No": 0})
test["transfusion"] = test["transfusion"].map({"Yes": 1, "No": 0})

# Convert DataFrame directly to PyTorch tensors
X_train_tensor = torch.tensor(train_sample.drop(columns=["transfusion"]).values, dtype=torch.float32)
y_train_tensor = torch.tensor(train_sample["transfusion"].values, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(test.drop(columns=["transfusion"]).values, dtype=torch.float32)
y_test_tensor = torch.tensor(test["transfusion"].values, dtype=torch.float32).unsqueeze(1)

# Create DataLoader for training
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


## Defining the Model
Here we will use an MLP with an input size of 80 as there are 80 predictors. Then two hidden layers of size 128 and 64, and an output layer of 1 for binary classification.

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.1)

        self.fc4 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()  # Use BCEWithLogitsLoss if you remove this

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

# Training
Now we train the MLP on the sample we made.

In [35]:
# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss, and optimizer
input_size = X_train_tensor.shape[1]
model = MLP(input_size).to(device)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move to GPU if available
        
        optimizer.zero_grad()  # Clear previous gradients
        outputs = model(X_batch)  # Forward pass
        loss = criterion(outputs, y_batch)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1/20, Loss: 0.2300
Epoch 2/20, Loss: 0.2255
Epoch 3/20, Loss: 0.2238
Epoch 4/20, Loss: 0.2226
Epoch 5/20, Loss: 0.2218
Epoch 6/20, Loss: 0.2210
Epoch 7/20, Loss: 0.2204
Epoch 8/20, Loss: 0.2199
Epoch 9/20, Loss: 0.2196
Epoch 10/20, Loss: 0.2191
Epoch 11/20, Loss: 0.2185
Epoch 12/20, Loss: 0.2186
Epoch 13/20, Loss: 0.2183
Epoch 14/20, Loss: 0.2178
Epoch 15/20, Loss: 0.2177
Epoch 16/20, Loss: 0.2173
Epoch 17/20, Loss: 0.2168
Epoch 18/20, Loss: 0.2167
Epoch 19/20, Loss: 0.2166
Epoch 20/20, Loss: 0.2163


In [ ]:
# Evaluate Model
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # No gradient computation for inference
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)0

    # Forward pass
    predictions = model(X_test_tensor)

    # Convert probabilities to binary labels (threshold at 0.5)
    pred_labels = (predictions > 0.5).float()

    # Move tensors back to CPU for sklearn evaluation
    y_true = y_test_tensor.cpu().numpy()
    y_pred_probs = predictions.cpu().numpy()  # Probabilities for AUROC
    y_pred_labels = pred_labels.cpu().numpy()  # Binary labels for accuracy

    # Compute Accuracy and AUROC
    accuracy = accuracy_score(y_true, y_pred_labels)
    auroc = roc_auc_score(y_true, y_pred_probs)  # Uses probabilities, not labels

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUROC: {auroc:.4f}")


Test Accuracy: 0.9204
Test AUROC: 0.8472
